In [1]:
def check_easy_case(number):
    """
    Check for some easy cases.
    """
    if is_prime(number):
        return number

    elif number.is_perfect_power():
        return number.perfect_power()[0]
    
    elif number % 2 == 0:
        return -1
    
    else:
        return 1

In [2]:
def get_degree(number):
    """
    Return the degree of the polynomial we use
    """
    digits_number = len(str(number))
    if digits_number < 60:
        return 3
    
    elif digits_number < 100:
        return 4
    
    else:
        return int((3*ln(number)/ln(ln(number)))^(1/3))

In [3]:
def get_m(number, degree):
    """
    Return number m that f(m) = 0 (mod n)
    """
    return int(number^(1/degree))

In [4]:
def get_polynomial(number, m):
    """
    Return the degree d polynomial f that f(m) = 0 (mod n) 
    """
    polynomial_array = number.digits(m)
    R.<x> = ZZ[]
    polynomial = R(polynomial_array)

    if polynomial.is_irreducible() == 0:
        return 0 # skip the early exit
    
    return polynomial

In [5]:
def get_bases_bound(number):
    """
    Return the bases bound
    """
    bound = (exp((8/9)^(1/3)*(log(number))^(1/3)*(log(log(number)))^(2/3)))
    
    return int(bound)

In [6]:
def get_rational_base(bound):
    """
    Return the set of all prime numbers less than the base bound
    """
    return prime_range(bound)

In [7]:
def get_algebraic_base(bound, polynomial):
    """
    Return the set of tuples (p, r) satisfying the Theorem 2.18
    """
    algebraic_base = []
    
    for p in prime_range(bound):
        for r in polynomial.change_ring(GF(p)).roots(None, False):
            algebraic_base.append((p, r.lift()))
            
    return algebraic_base

In [8]:
def get_quad_char_base(number, bound, polynomial):
    """
    Return the set of tuples (q, s) representing quadratic characters
    """
    quad_char_base = []
    quad_char_num = min(max(10, 3 * len(str(n))), 100)
    q = next_prime(int(bound))
    g = derivative(polynomial)
    
    while len(quad_char_base) < quad_char_num:
        for s in polynomial.change_ring(GF(q)).roots(None, False):
            if s not in g.change_ring(GF(q)).roots(None, False):
                quad_char_base.append((q,s))
                break
        q = next_prime(q)
        
    return quad_char_base

In [9]:
def get_norm(a, b, polynomial):
    """
    Return the value that is norm of an algebraic integer a - b * alpha in QQ(alpha)
    """
    return ((-b)^(polynomial.degree()) * polynomial.subs(x = (-a/b)))

In [10]:
def check_smoothness(a, b, m, polynomial, rational_base, algebraic_base):
    """
    Check for the B-smoothness of rational integer (a + b * m) and ideal (a + b * alpha)
    """
    if a != 0 and gcd(a, b) == 1:
        actual_num = (a + b * m) * get_norm(a, b, polynomial)
        if actual_num == 0:
            return 0

        for num in rational_base:
            while actual_num % num == 0:
                actual_num = actual_num / num

        for pair in algebraic_base:
            while actual_num % pair[0]  == 0:
                actual_num = actual_num / pair[0]
                
        if actual_num == 1 or actual_num == - 1:
            return 1
    return 0

In [11]:
def get_rat_factor_power(number, prime):
    """
    Return the biggest power that a rational integer divided by a prime
    """
    count = 0
    
    while number % prime == 0:
        number = number / prime
        count += 1
        
    return count

In [12]:
def get_alg_factor_power(a, b, p, r, polynomial):
    """
    Return the biggest power that an algebraic integer or an ideal <a + b alpha> divided by an ideal <p, alpha - r>
    """
    count = 0
    check_num = get_norm(a, b, polynomial)
    
    if (a + b * r) % p == 0:
        if b % p != 0:
            while check_num % p == 0:
                check_num = check_num / p
                count += 1
        return count
    return count

In [13]:
def get_smooth_pairs(number, V, m, polynomial, rational_base, algebraic_base):
    """
    Sieving process: find more than V (a, b) pairs satisfying the smoothness in a reasonable sieving interval
    """
    sieving_interval = max(500, ceil(log(number) * 500 / log(10)))
    pairs_array = []
    b = 1
    
    while b >= 1:
        for a in range(-sieving_interval, sieving_interval + 1):
            if check_smoothness(a, b, m, polynomial, rational_base, algebraic_base) == 1:
                pairs_array.append((a, b))
                if len(pairs_array) > V:
                    return pairs_array
        b += 1

In [14]:
def get_matrix_A(pairs_array, m, rational_base, algebraic_base, quad_char_base, polynomial, V):
    """
    Return the matrix A with rows are the exponential vectors representing each (a, b) pairs
    """
    ex_vectors_array = [] 
    
    for tuple in pairs_array:
        check_rational_num = tuple[0] + tuple[1] * m 
        check_algebraic_num = get_norm(tuple[0], tuple[1], polynomial) 
        
        ex_vector = [0 for x in range(V)]

        if check_rational_num < 0:
            ex_vector[0] = 1

        for i in range(len(rational_base)):
            ex_vector[1 + i] = get_rat_factor_power(check_rational_num, rational_base[i])

        for j in range(len(algebraic_base)):
            ex_vector[1 + len(rational_base) + j] = get_alg_factor_power(tuple[0], tuple[1], algebraic_base[j][0], algebraic_base[j][1], polynomial)

        for k in range(len(quad_char_base)):
            if legendre_symbol(tuple[0] + tuple[1] * quad_char_base[k][1], quad_char_base[k][0]) == -1:
                ex_vector[1 + len(rational_base) + len(algebraic_base) + k] = 1
                
        ex_vectors_array.append(ex_vector)
        matrix_A = matrix(GF(2), ex_vectors_array)
        
    return matrix_A

In [15]:
def get_solutions_set(matrix_A):
    """
    Return the solutions set of the equation x^T * A = 0 (mod 2)
    """
    return matrix_A.left_kernel().basis_matrix()

In [16]:
def get_set_S(solution, pairs_array):
    """
    Return the set S of (a, b) pairs that satisfying the square conditions in ZZ/nZZ and ZZ[alpha]
    """
    S = []
    
    for i in range(len(solution)):
        if solution[i] == 1:
            S.append(pairs_array[i])
            
    return S

In [17]:
def get_rational_product(S, polynomial, m):
    """
    Compute the rational product of (a + b * m)
    """
    u_square = (derivative(polynomial).subs(m))^2
    
    for tuple in S:
        u_square *= (tuple[0] + tuple[1] * m)
        
    return u_square

In [18]:
def get_rational_sqrt(u_square, number):
    """
    Compute rational squared root in ZZ/nZZ
    """
    u = u_square.sqrt()
    u = mod(u, number)
    
    return u

In [19]:
def get_algebraic_product(S, polynomial):
    """
    Compute the algebraic product of ideals in Z[alpha]
    """
    Ka = PolynomialRing(ZZ, "a")
    gamma = Ka(derivative(polynomial)^2)
    
    for tuple in S:
        gamma *= Ka([tuple[0], 1 * tuple[1]])
    gamma = gamma.mod(Ka(polynomial))
    
    return gamma

In [20]:
def get_algebraic_sqrt(S, polynomial, gamma):
    """
    Return the algebraic squared root in ZZ[alpha]
    """
    K.<a> = NumberField(polynomial)
    gamma = gamma(a)
    
    if gamma.is_square():
        return gamma.sqrt()

In [21]:
def get_v(beta, m, number):
    """
    Return the number v using the sigma homomorphism
    """
    v = beta.lift().subs(x = m)
    v = v % number
    
    return v

In [22]:
def get_factors(u, v, number):
    """
    Return the factors
    """
    result = (u - v).lift()
    
    return gcd(result, number)

In [23]:
def factor_by_NFS(number):
    print("Factoring: ", number)
    results_set = []
    if check_easy_case(number) == -1:
        results_set.add(2)
        while number % 2 == 0:
            number = number / 2
    elif check_easy_case(number) == 1:
        degree = get_degree(number)
        m = get_m(number, degree)
        print("\nm: ", m)
        polynomial = get_polynomial(number, m)
        print("\nPolynomial using: ", polynomial)
        bases_bound = get_bases_bound(number)
        print("\nBases bound: ", bases_bound)
        rational_base = get_rational_base(bases_bound)
        print("\nRational base: ", rational_base)
        print("\nlength: ", len(rational_base))
        algebraic_base = get_algebraic_base(bases_bound, polynomial)
        print("\nAlgebraic base: ", algebraic_base)
        print("\nlength: ", len(algebraic_base))
        quad_char_base = get_quad_char_base(number, bases_bound, polynomial)
        print("\nQuadratic characters base: ", quad_char_base)
        print("\nlength: ", len(quad_char_base))
        V = 1 + len(rational_base) + len(algebraic_base) + len(quad_char_base)
        print("\nNumber of pairs to find: ", V + 1)
        pairs_array = get_smooth_pairs(number, V, m, polynomial, rational_base, algebraic_base)
        print("\n(a, b) pairs are B-smooth: ", pairs_array)
        matrix_A = get_matrix_A(pairs_array, m, rational_base, algebraic_base, quad_char_base, polynomial, V)
        print("\nMatrix A:")
        print(matrix_A)
        solutions_set = get_solutions_set(matrix_A)
        print("\nSolution set: ")
        print(solutions_set)
        for solution in solutions_set.rows():
            S = get_set_S(solution, pairs_array)
            u_square = get_rational_product(S, polynomial, m)
            u = get_rational_sqrt(u_square, number)
            gamma = get_algebraic_product(S, polynomial)
            beta = get_algebraic_sqrt(S, polynomial, gamma)
            v = get_v(beta, m, number)
            factor_result = get_factors(u, v, number)
            if 1 < factor_result < number and factor_result.is_prime():
                if factor_result not in results_set:
                    print("-"*30)
                    print("\nA solution satisfied square conditions: ")
                    print(solution)
                    print("\nThe set S: ")
                    print(S)
                    print("\nu square: ", u_square)
                    print("\nu: ", u)
                    print("\nideal beta square: ", gamma)
                    print("\nideal beta: ", beta)
                    print("\n v: ", v)
                    print("\nPrime factor found: ", factor_result)
                    results_set.append(factor_result)
        print("-"*30)
    return results_set